In [1]:
from p2pfl.node import Node
from p2pfl.learning.pytorch.mnist_examples.mnistfederated_dm import MnistFederatedDM
from p2pfl.learning.pytorch.mnist_examples.models.mlp import MLP
from collections import OrderedDict
import torch
import time

def test_convergence(x):
    n,r = x

    # Node Creation
    nodes = []
    for i in range(n):
        node = Node(MLP(),MnistFederatedDM())
        node.start()
        nodes.append(node)

    # Node Connection
    for i in range(len(nodes)-1):
        nodes[i+1].connect_to(nodes[i].host,nodes[i].port)
        time.sleep(0.1)

    # Check if they are connected
    for node in nodes:
        assert len(node.neightboors) == n-1

    # Start Learning
    nodes[0].set_start_learning(rounds=r,epochs=0)

    # Wait 4 results
    while True:
        time.sleep(1)
        finish = True
        for f in [node.round is None for node in nodes]:
            finish = finish and f

        if finish:
            break


    # Validamos Modelos obtenidos sean iguales
    model = None
    first = True
    for node in nodes:
        if first:
            model = node.learner.get_parameters()
            first = False
        else:
            for layer in model:
                a = torch.round(model[layer], decimals=2)
                b = torch.round(node.learner.get_parameters()[layer], decimals=2)
                assert torch.eq(a, b).all()

    # Cerrar
    for node in nodes:
        node.stop()
        time.sleep(.1) #Esperar por la asincronía
    
    return nodes
        
nodes = test_convergence((4,2))

Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 60179
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 59649
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 37499
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 40713
INFO:root:(('127.0.0.1', 60179)) Conexión aceptada con ('127.0.0.1', 59649)
INFO:root:(('127.0.0.1', 59649)) Conexión aceptada con ('127.0.0.1', 37499)
INFO:root:(('127.0.0.1', 37499)) Conexión aceptada con ('127.0.0.1', 60179)
INFO:root:(('127.0.0.1', 37499)) Conexión aceptada con ('127.0.0.1', 40713)
INFO:root:(('127.0.0.1', 40713)) Conexión aceptada con ('127.0.0.1', 59649)
INFO:root:(('127.0.0.1', 40713)) Conexión aceptada con ('127.0.0.1', 60179)
INFO:root:(('127.0.0.1', 60179)) Broadcasting start learning...
INFO:root:(('127.0.0.1', 59649)) Broadcasting Number of Samples...
INFO:root:(('127.0.0.1', 37499)) Broadcasting Number of Samples...
INFO:root:(('127.0.0.1', 6

In [3]:
from p2pfl.learning.pytorch.mnist_examples.mnistfederated_dm import MnistFederatedDM
from p2pfl.learning.pytorch.mnist_examples.models.cnn import CNN
from p2pfl.learning.pytorch.mnist_examples.models.mlp import MLP
from p2pfl.node import Node
import pytest
import time
import threading

nodes = []
        
def test_node_down_on_learning(n):

    # Node Creation
    nodes = []
    for i in range(n):
        node = Node(MLP(),MnistFederatedDM())
        node.start()
        nodes.append(node)

    # Node Connection
    for i in range(len(nodes)-1):
        nodes[i+1].connect_to(nodes[i].host,nodes[i].port)
        time.sleep(1)

    # Check if they are connected
    for node in nodes:
        assert len(node.neightboors) == n-1

    # Start Learning
    nodes[0].set_start_learning(rounds=2,epochs=0)

    # Stopping node
    nodes[1].stop()
    # Wait 4 results
    while True:
        time.sleep(1)
        finish = True
        for f in [node.round is None for node in nodes]:
            finish = finish and f

        if finish:
            break

    for node in nodes:
        node.stop()


nodes = []
for _ in range(6):
    test_node_down_on_learning(8)


Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 58345
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 58087
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 35365
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 36109
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 45637
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 51189
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 44467
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 57173
INFO:root:(('127.0.0.1', 58345)) Conexión aceptada con ('127.0.0.1', 58087)
INFO:root:(('127.0.0.1', 58087)) Conexión aceptada con ('127.0.0.1', 35365)
INFO:root:(('127.0.0.1', 35365)) Conexión aceptada con ('127.0.0.1', 58345)
INFO:root:(('127.0.0.1', 35365)) Conexión aceptada con ('127.0.0.1', 36109)
INFO:root:(('127.0.0.1', 36109)) Conexión aceptada con (

INFO:root:(127.0.0.1:58345) Model added (5/7) from ('127.0.0.1', 35365)
INFO:root:(127.0.0.1:51189) Model added (5/7) from ('127.0.0.1', 35365)
INFO:root:(127.0.0.1:57173) Model added (5/7) from ('127.0.0.1', 45637)
INFO:root:(127.0.0.1:45637) Model added (5/7) from ('127.0.0.1', 57173)
INFO:root:(127.0.0.1:44467) Model added (6/7) from ('127.0.0.1', 35365)
INFO:root:(127.0.0.1:36109) Model added (5/7) from ('127.0.0.1', 35365)
INFO:root:(127.0.0.1:35365) Model added (6/7) from ('127.0.0.1', 57173)
INFO:root:(127.0.0.1:58345) Model added (6/7) from ('127.0.0.1', 57173)
INFO:root:(127.0.0.1:51189) Model added (6/7) from ('127.0.0.1', 57173)
INFO:root:(127.0.0.1:57173) Model added (6/7) from ('127.0.0.1', 35365)
INFO:root:(127.0.0.1:45637) Model added (6/7) from ('127.0.0.1', 35365)
INFO:root:(127.0.0.1:44467) Model added (7/7) from ('127.0.0.1', 57173)
INFO:root:(127.0.0.1:36109) Model added (6/7) from ('127.0.0.1', 57173)
INFO:root:(127.0.0.1:35365) Model added (7/7) from ('127.0.0.1',

INFO:root:(('127.0.0.1', 57173)) Finish!!.
INFO:root:(('127.0.0.1', 45637)) Finish!!.
INFO:root:Bajando el nodo, dejando de escuchar en 127.0.0.1 58345 y desconectándose de 6 nodos
INFO:root:Bajando el nodo, dejando de escuchar en 127.0.0.1 35365 y desconectándose de 6 nodos
DEBUG:root:Closed connection: ('127.0.0.1', 58345)
INFO:root:Bajando el nodo, dejando de escuchar en 127.0.0.1 36109 y desconectándose de 6 nodos
DEBUG:root:Closed connection: ('127.0.0.1', 35365)
DEBUG:root:Closed connection: ('127.0.0.1', 58345)
DEBUG:root:Closed connection: ('127.0.0.1', 58345)
DEBUG:root:Closed connection: ('127.0.0.1', 35365)
DEBUG:root:Closed connection: ('127.0.0.1', 35365)
DEBUG:root:Closed connection: ('127.0.0.1', 35365)
INFO:root:Bajando el nodo, dejando de escuchar en 127.0.0.1 45637 y desconectándose de 6 nodos
DEBUG:root:Closed connection: ('127.0.0.1', 35365)
DEBUG:root:Closed connection: ('127.0.0.1', 58345)
Train: 54000 Val:6000 Test:10000DEBUG:root:Closed connection: ('127.0.0.1',

DEBUG:root:Closed connection: ('127.0.0.1', 58809)
INFO:root:(127.0.0.1:48897) Model added (1/7) from ('127.0.0.1', 47017)
INFO:root:(127.0.0.1:45023) Model added (1/7) from ('127.0.0.1', 47017)
INFO:root:(127.0.0.1:48119) Model added (1/7) from ('127.0.0.1', 47017)
INFO:root:(127.0.0.1:55363) Model added (1/7) from ('127.0.0.1', 47017)
INFO:root:(127.0.0.1:44381) Model added (1/7) from ('127.0.0.1', 47017)
INFO:root:(127.0.0.1:58809) Model added (1/7) from ('127.0.0.1', 47017)
INFO:root:(('127.0.0.1', 48119)) Training...
INFO:root:(127.0.0.1:48119) Model added (2/7) from ('127.0.0.1', 48119)
INFO:root:(('127.0.0.1', 48897)) Training...
INFO:root:(('127.0.0.1', 48119)) Broadcasting model to 6 clients. (size: 548864 bytes)
INFO:root:(('127.0.0.1', 45023)) Training...
INFO:root:(127.0.0.1:48897) Model added (2/7) from ('127.0.0.1', 48897)
INFO:root:(('127.0.0.1', 44381)) Training...
INFO:root:(127.0.0.1:45023) Model added (2/7) from ('127.0.0.1', 45023)
INFO:root:(('127.0.0.1', 55363)) T

INFO:root:(127.0.0.1:55363) Model added (5/7) from ('127.0.0.1', 48119)
INFO:root:(127.0.0.1:58809) Model added (4/7) from ('127.0.0.1', 48119)
INFO:root:(127.0.0.1:45023) Model added (4/7) from ('127.0.0.1', 58809)
INFO:root:(127.0.0.1:48119) Model added (5/7) from ('127.0.0.1', 44381)
INFO:root:(127.0.0.1:44381) Model added (5/7) from ('127.0.0.1', 48119)
INFO:root:(127.0.0.1:48897) Model added (5/7) from ('127.0.0.1', 48119)
INFO:root:(127.0.0.1:47017) Model added (5/7) from ('127.0.0.1', 55363)
INFO:root:(127.0.0.1:55363) Model added (6/7) from ('127.0.0.1', 44381)
INFO:root:(127.0.0.1:58809) Model added (5/7) from ('127.0.0.1', 44381)
INFO:root:(127.0.0.1:45023) Model added (5/7) from ('127.0.0.1', 44381)
INFO:root:(127.0.0.1:48119) Model added (6/7) from ('127.0.0.1', 47017)
INFO:root:(127.0.0.1:44381) Model added (6/7) from ('127.0.0.1', 55363)
INFO:root:(127.0.0.1:48897) Model added (6/7) from ('127.0.0.1', 55363)
INFO:root:(127.0.0.1:47017) Model added (6/7) from ('127.0.0.1',

...

DEBUG:root:Closed connection: ('127.0.0.1', 46665)
DEBUG:root:Closed connection: ('127.0.0.1', 46665)
DEBUG:root:Closed connection: ('127.0.0.1', 35437)
INFO:root:(('127.0.0.1', 35437)) Broadcasting model to 6 clients. (size: 548864 bytes)
DEBUG:root:Closed connection: ('127.0.0.1', 46665)
DEBUG:root:Closed connection: ('127.0.0.1', 46665)
DEBUG:root:Closed connection: ('127.0.0.1', 57929)
DEBUG:root:Closed connection: ('127.0.0.1', 54275)
DEBUG:root:Closed connection: ('127.0.0.1', 47027)
DEBUG:root:Closed connection: ('127.0.0.1', 53645)
DEBUG:root:Closed connection: ('127.0.0.1', 45875)
DEBUG:root:Closed connection: ('127.0.0.1', 33059)
INFO:root:(127.0.0.1:53645) Model added (1/7) from ('127.0.0.1', 35437)
INFO:root:(127.0.0.1:33059) Model added (1/7) from ('127.0.0.1', 35437)
INFO:root:(127.0.0.1:47027) Model added (1/7) from ('127.0.0.1', 35437)
INFO:root:(127.0.0.1:54275) Model added (1/7) from ('127.0.0.1', 35437)
INFO:root:(127.0.0.1:57929) Model added (1/7) from ('127.0.0.1',

INFO:root:(127.0.0.1:47027) Model added (4/7) from ('127.0.0.1', 53645)
INFO:root:(127.0.0.1:33059) Model added (3/7) from ('127.0.0.1', 57929)
INFO:root:(127.0.0.1:54275) Model added (3/7) from ('127.0.0.1', 47027)
INFO:root:(127.0.0.1:57929) Model added (3/7) from ('127.0.0.1', 53645)
INFO:root:(127.0.0.1:53645) Model added (4/7) from ('127.0.0.1', 47027)
INFO:root:(127.0.0.1:45875) Model added (4/7) from ('127.0.0.1', 53645)
INFO:root:(127.0.0.1:35437) Model added (4/7) from ('127.0.0.1', 47027)
INFO:root:(127.0.0.1:47027) Model added (5/7) from ('127.0.0.1', 45875)
INFO:root:(127.0.0.1:33059) Model added (4/7) from ('127.0.0.1', 53645)
INFO:root:(127.0.0.1:54275) Model added (4/7) from ('127.0.0.1', 53645)
INFO:root:(127.0.0.1:57929) Model added (4/7) from ('127.0.0.1', 47027)
INFO:root:(127.0.0.1:53645) Model added (5/7) from ('127.0.0.1', 45875)
INFO:root:(127.0.0.1:45875) Model added (5/7) from ('127.0.0.1', 47027)
INFO:root:(127.0.0.1:35437) Model added (5/7) from ('127.0.0.1',

INFO:root:(('127.0.0.1', 49183)) Conexión aceptada con ('127.0.0.1', 53663)
INFO:root:(('127.0.0.1', 53663)) Conexión aceptada con ('127.0.0.1', 54991)
INFO:root:(('127.0.0.1', 53663)) Conexión aceptada con ('127.0.0.1', 43041)
INFO:root:(('127.0.0.1', 53663)) Conexión aceptada con ('127.0.0.1', 45593)
INFO:root:(('127.0.0.1', 53663)) Conexión aceptada con ('127.0.0.1', 57517)
INFO:root:(('127.0.0.1', 53663)) Conexión aceptada con ('127.0.0.1', 44979)
INFO:root:(('127.0.0.1', 53663)) Conexión aceptada con ('127.0.0.1', 60793)
INFO:root:(('127.0.0.1', 60793)) Conexión aceptada con ('127.0.0.1', 49183)
INFO:root:(('127.0.0.1', 60793)) Conexión aceptada con ('127.0.0.1', 54991)
INFO:root:(('127.0.0.1', 60793)) Conexión aceptada con ('127.0.0.1', 45593)
INFO:root:(('127.0.0.1', 60793)) Conexión aceptada con ('127.0.0.1', 43041)
INFO:root:(('127.0.0.1', 60793)) Conexión aceptada con ('127.0.0.1', 57517)
INFO:root:(('127.0.0.1', 60793)) Conexión aceptada con ('127.0.0.1', 44979)
INFO:root:((

NOT IMPLEMENTED---------plantearlo bien------------------------------------------------------------------------------------------
INFO:root:(127.0.0.1:53663) Agregating models. Timeout reached
NOT IMPLEMENTED---------plantearlo bien------------------------------------------------------------------------------------------
INFO:root:(('127.0.0.1', 49183)) Waiting other nodes.
INFO:root:(127.0.0.1:60793) Agregating models. Timeout reached
NOT IMPLEMENTED---------plantearlo bien------------------------------------------------------------------------------------------
INFO:root:(('127.0.0.1', 54991)) Waiting other nodes.
INFO:root:(('127.0.0.1', 53663)) Waiting other nodes.
INFO:root:(('127.0.0.1', 53663)) Round 1 of 2 finished.
INFO:root:(('127.0.0.1', 54991)) Round 1 of 2 finished.
INFO:root:(('127.0.0.1', 49183)) Round 1 of 2 finished.
INFO:root:(('127.0.0.1', 53663)) Training...
INFO:root:(('127.0.0.1', 45593)) Round 1 of 2 finished.
INFO:root:(('127.0.0.1', 60793)) Waiting other nodes.

INFO:root:Bajando el nodo, dejando de escuchar en 127.0.0.1 60793 y desconectándose de 6 nodos
DEBUG:root:Closed connection: ('127.0.0.1', 57517)
DEBUG:root:Closed connection: ('127.0.0.1', 49183)
DEBUG:root:Closed connection: ('127.0.0.1', 54991)
DEBUG:root:Closed connection: ('127.0.0.1', 54991)
DEBUG:root:Closed connection: ('127.0.0.1', 53663)
DEBUG:root:Closed connection: ('127.0.0.1', 60793)
DEBUG:root:Closed connection: ('127.0.0.1', 49183)
DEBUG:root:Closed connection: ('127.0.0.1', 53663)
DEBUG:root:Closed connection: ('127.0.0.1', 60793)
DEBUG:root:Closed connection: ('127.0.0.1', 54991)
DEBUG:root:Closed connection: ('127.0.0.1', 54991)
DEBUG:root:Closed connection: ('127.0.0.1', 54991)
DEBUG:root:Closed connection: ('127.0.0.1', 54991)
DEBUG:root:Closed connection: ('127.0.0.1', 57517)
DEBUG:root:Closed connection: ('127.0.0.1', 57517)
DEBUG:root:Closed connection: ('127.0.0.1', 57517)
DEBUG:root:Closed connection: ('127.0.0.1', 49183)
DEBUG:root:Closed connection: ('127.0.

In [2]:
#import threading
for thread in threading.enumerate(): 
    print(thread.name)

MainThread
IOPub
Heartbeat
Thread-3
Thread-4
Control
IPythonHistorySavingThread
Thread-2


In [6]:
import sys

def get_thread_position(thread):
    frame = sys._current_frames().get(thread.ident, None)
    if frame:
        return frame.f_code.co_filename, frame.f_code.co_name, frame.f_code.co_firstlineno

get_thread_position(threading.enumerate()[-1])

('/Users/pedro/Documents/UDC/4/TFG/federated_learning_p2p/p2pfl/node.py',
 '__on_round_finished',
 260)

In [1]:
from p2pfl.const import HEARTBEAT_FREC, SOCKET_TIEMOUT
from p2pfl.learning.pytorch.mnist_examples.mnistfederated_dm import MnistFederatedDM
from p2pfl.learning.pytorch.mnist_examples.models.cnn import CNN
from p2pfl.learning.pytorch.mnist_examples.models.mlp import MLP
from p2pfl.node import Node
import pytest
import time
from test.fixtures import two_nodes, four_nodes


import threading
for thread in threading.enumerate(): 
    print(thread.name)
    
n1 = Node(MLP(),MnistFederatedDM())
n2 = Node(MLP(),MnistFederatedDM())
print("\n")
def test_bad_binary_model():

    n1.start()
    n2.start()

    n1.connect_to(n2.host,n2.port)
    time.sleep(0.1) 

    # Start Learning
    n1.set_start_learning(rounds=2,epochs=0)
 
    # Adding noise to the buffer
    for _ in range(200):
        n1.neightboors[0].param_bufffer += "noise".encode("utf-8")
    
    while not n1.round is None and not n2.round is None:
        time.sleep(0.1)
        
    n1.stop()
    n2.stop()
    
test_bad_binary_model()

time.sleep(10)

for thread in threading.enumerate(): 
    print(thread.name)
    
print("\n")



MainThread
IOPub
Heartbeat
Thread-3
Thread-4
Control
IPythonHistorySavingThread
Thread-2
Train: 54000 Val:6000 Test:10000
Train: 54000 Val:6000 Test:10000


INFO:root:Nodo a la escucha en 127.0.0.1 54998
INFO:root:Nodo a la escucha en 127.0.0.1 54999
INFO:root:(('127.0.0.1', 54999)) Conexión aceptada con ('127.0.0.1', 54998)
INFO:root:(('127.0.0.1', 54998)) Broadcasting start learning...
INFO:root:(('127.0.0.1', 54998)) Sending Initial Model Weights
INFO:root:(('127.0.0.1', 54999)) Initialicing Model Weights
INFO:root:(('127.0.0.1', 54998)) Broadcasting model to 1 clients. (size: 548864 bytes)
INFO:root:(('127.0.0.1', 54998)) Broadcasting Number of Samples...
INFO:root:(('127.0.0.1', 54998)) Training...
INFO:root:(('127.0.0.1', 54999)) Model initialized
INFO:root:(('127.0.0.1', 54998)) Model added (1/2) from ('127.0.0.1', 54998)
INFO:root:(('127.0.0.1', 54998)) Broadcasting model to 1 clients. (size: 548864 bytes)
INFO:root:(('127.0.0.1', 54999)) Model added (1/2) from ('127.0.0.1', 54

In [4]:

for thread in threading.enumerate(): 
    print(thread.name)
    

MainThread
IOPub
Heartbeat
Thread-3
Thread-4
Control
IPythonHistorySavingThread
Thread-2


In [6]:
print(n1.round)

None
